In [1858]:
import pandas as pd
from util.performance_metrics import *
 
url =  "https://raw.githubusercontent.com/xuda1979/DF/master/sp500_companies/data/constituents.csv"
df_symbols =pd.read_csv('./sp500_companies/data/constituents.csv')

In [41]:
df_symbols.head()
all_symbols = list(df_symbols['Symbol'].unique())
all_symbols.append('SPY')

In [42]:
# Import yfinance
import yfinance as yf  
 
# Get the data for the stock Apple by specifying the stock ticker, start date, and end date
data_all =pd.DataFrame({})
for symbol in all_symbols:
    
    data = yf.download(symbol)#,'2016-01-01','2018-01-01')
    data['Symbol'] = symbol
    if(len(data)>0):
        data_all = pd.concat([data_all,data])
 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HRS: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HCP: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%****************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TMK: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TSS: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%****************

In [44]:
symbols = list(data_all['Symbol'].unique())

In [646]:
data_all = data_all.reset_index(drop=True)
 

In [1860]:
data_all.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1970-01-02,6.851562,6.890625,6.843750,6.851562,1.483333,72000.0,MMM
1,1970-01-05,6.859375,6.898438,6.859375,6.890625,1.491789,446400.0,MMM
2,1970-01-06,6.890625,6.960938,6.882812,6.960938,1.507011,176000.0,MMM
3,1970-01-07,6.960938,7.015625,6.945312,7.000000,1.515469,164800.0,MMM
4,1970-01-08,7.000000,7.109375,6.984375,7.093750,1.535765,304000.0,MMM


In [1861]:
training_start_date = '2015-01-15'
training_end_date = '2020-09-30'
mask = data_all['Date'] >= training_start_date
mask &= data_all['Date'] <= training_end_date

In [1862]:
data_training= data_all[mask] 

In [816]:
training_symbols = set({})
history_len = []
for symbol in symbols:
    mask = data_training['Symbol'] == symbol
    history_len.append(len(data_training[mask]['Date'].unique()))
for symbol in symbols:
    mask = data_training['Symbol'] == symbol
 
    
    if len(data_training[mask]['Date'].unique()) >= np.mean(history_len)*0.99:
        training_symbols.add(symbol)
        

In [823]:
def date_filter(df, start_date, end_date):
    mask = df['Date'] >=  start_date
    mask &= df['Date'] <= end_date
    return df[mask]

In [931]:
def covariance(df, date, symbols, history):
    df_recent = pd.DataFrame({})
    dates = list(df['Date'].unique())
    dates.sort()
    df_recent['Date']= dates[-(history+1):-1]
    for symbol in symbols:
        mask = df['Symbol'] == symbol
 
        if all([date in list(df[mask]['Date'].unique()) for date in dates[-(history+1):-1]]):
            #print(df[mask]['Close'][-(history+1):-1])
            df_recent[symbol] = df[mask]['Close'].to_list()[-(history+1):-1]
    df_return = pd.DataFrame({})
    df_return['Date'] = df_recent['Date']
    for symbol in df_recent.columns:
        if symbol !='Date':
            #print(symbol,df_recent[symbol].div(df_recent[symbol].shift(1)))
            df_return[symbol] = df_recent[symbol].div(df_recent[symbol].shift(1))
    return df_return[1:].set_index('Date').cov() 
 
            
symbols = ['AAPL', 'MMM']
df_cov = covariance(data_training, list(data_training['Date'].unique())[-1], symbols, 10)
cov=df_cov.to_numpy() 
print(cov)

[[7.85042934e-04 7.44584306e-06]
 [7.44584306e-06 3.88371563e-04]]


In [988]:
def objective(x):
    l = len(x)
    assert l == cov.shape[0]
    return x.dot(cov.dot(x))

def constraint(x):
    return np.atleast_1d(np.sum(np.abs(x)) - 1.5)

 
bounds=[[0, 1], [1, 2]]
res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )
res.x    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0006603349124938092
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2


<ipython-input-988-e88c3116b488>:11: OptimizeWarning: Unknown solver options: gtol
  res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )


array([0.25038093, 1.24961907])

In [1025]:
from scipy.optimize import minimize
def portfolio_optimization(cov, k):        
 
    assert 2*k == cov.shape[0]
    def objective(x):
        return x.dot(cov.dot(x))

    def constraint(x):
        return np.atleast_1d(np.sum(np.abs(x))-1)
    x0 = [1/(2*k)]*k+[-1/(2*k)]*k
    x0 = np.array(x0)
    bounds = []
    for i in range(k):
        bounds.append([1/(4*k),1/k])
    for i in range(k):
        bounds.append([-1/k, -1/(4*k)])
        
    res=optimize.minimize(objective, x0, constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': False}, bounds=bounds )
    s= sum(np.abs(res.x))
    return res.x/s 

# Factor model  

#### Genearte DataFrames of various returns for each symbol 

In [1822]:
data_training_return = {}
 
for symbol in symbols:
    print(symbol)
    mask = data_training['Symbol'] == symbol
    data_training_return[symbol] = pd.DataFrame({})
    data_training_return[symbol]['Date'] = data_training[mask]['Date']
    
    data_training_return[symbol]['close_close_return']  = data_training[mask]['Close'].div(data_training[mask]['Close'].shift(1))-1  
    data_training_return[symbol]['open_close_return']  = data_training[mask]['Close'].div(data_training[mask]['Open'] )-1
    data_training_return[symbol]['close_open_return']  = data_training[mask]['Open'].div(data_training[mask]['Close'].shift(1) )-1
    data_training_return[symbol]['vwap_open_return']  = 4*data_training[mask]['Open'].div(data_training[mask]['Open'].shift(1)+ \
                                                                                        data_training[mask]['High'].shift(1)+ \
                                                                                        data_training[mask]['Low'].shift(1)+ \
                                                                                        data_training[mask]['Close'].shift(1))-1
    
    data_training_return[symbol]['vwap_close_return']  = 4*data_training[mask]['Open'].div(data_training[mask]['Open']+ \
                                                                                        data_training[mask]['High']+ \
                                                                                        data_training[mask]['Low']+ \
                                                                                        data_training[mask]['Close'])-1
    
    
    data_training_return[symbol].fillna(0, inplace=True)
    data_training_return[symbol].reset_index(inplace=True,drop=True)
    
   


MMM
AOS
ABT
ABBV
ACN
ATVI
AYI
ADBE
AAP
AMD
AES
AET
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANDV
ANSS
ANTM
AON
APA
AIV
AAPL
AMAT
APTV
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AZO
AVB
AVY
BLL
BAC
BAX
BDX
BBY
BIIB
BLK
HRB
BA
BWA
BXP
BSX
BHF
BMY
AVGO
CHRW
CA
COG
CDNS
CPB
COF
CAH
KMX
CCL
CAT
CBOE
CNC
CNP
CTL
CERN
CF
SCHW
CHTR
CHK
CVX
CMG
CB
CHD
CI
XEC
CINF
CTAS
CSCO
C
CFG
CTXS
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
GLW
COST
COTY
CCI
CSRA
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DOV
DTE
DUK
DRE
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EVHC
EOG
EQT
EFX
EQIX
EQR
ESS
EL
RE
ES
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FISV
FLIR
FLS
FLR
FMC
FL
F
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
GD
GE
GIS
GM
GPC
GILD
GPN
GS
GT
GWW
HAL
HBI
HOG
HIG
HAS
HCA
HP
HSIC
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HPQ
HUM
HBAN
HII
IDXX
INFO
ITW
ILMN
INCY
IR
INTC
ICE
IBM
IP
IPG
IFF
INTU
ISRG
IVZ
I

#### Train the regression model 

In [1851]:
testing_length = 250
training_dates= set(data_training_return['SPY']['Date'][:-testing_length])


import numpy as np
from sklearn.linear_model import LinearRegression
X = data_training_return['SPY']['close_close_return'] 
dates_X = set(data_training_return['SPY']['Date'] )
dates_X = dates_X.intersection(training_dates)

reg_map = {}
for symbol in symbols:
 
 
    
        dates_y = set(data_training_return[symbol]['Date'] )
        dates = dates_y.intersection(dates_X)
        mask = data_training_return[symbol]['Date'].isin(dates)
        y =  data_training_return[symbol][mask]['close_close_return'] 

        mask = data_training_return['SPY']['Date'].isin(dates)
        X_here = data_training_return['SPY'][mask]['close_close_return'] 
        X_here = np.expand_dims(X_here, axis=1)
        print(X_here.shape, y.shape, symbol)
        if X_here.shape[0] == len(dates_X):
            reg = LinearRegression().fit(X_here, y)
            reg_map[symbol] = reg
 
# reg.score(X, y)

# reg.coef_

# reg.intercept_
 
 

(473, 1) (473,) MMM
(473, 1) (473,) AOS
(473, 1) (473,) ABT
(473, 1) (473,) ABBV
(473, 1) (473,) ACN
(473, 1) (473,) ATVI
(473, 1) (473,) AYI
(473, 1) (473,) ADBE
(473, 1) (473,) AAP
(473, 1) (473,) AMD
(473, 1) (473,) AES
(222, 1) (222,) AET
(473, 1) (473,) AMG
(473, 1) (473,) AFL
(473, 1) (473,) A
(473, 1) (473,) APD
(473, 1) (473,) AKAM
(473, 1) (473,) ALK
(473, 1) (473,) ALB
(473, 1) (473,) ARE
(473, 1) (473,) ALXN
(473, 1) (473,) ALGN
(473, 1) (473,) ALLE
(473, 1) (473,) ADS
(473, 1) (473,) LNT
(473, 1) (473,) ALL
(473, 1) (473,) GOOGL
(473, 1) (473,) GOOG
(473, 1) (473,) MO
(473, 1) (473,) AMZN
(473, 1) (473,) AEE
(473, 1) (473,) AAL
(473, 1) (473,) AEP
(473, 1) (473,) AXP
(473, 1) (473,) AIG
(473, 1) (473,) AMT
(473, 1) (473,) AWK
(473, 1) (473,) AMP
(473, 1) (473,) ABC
(473, 1) (473,) AME
(473, 1) (473,) AMGN
(473, 1) (473,) APH
(473, 1) (473,) ADI
(180, 1) (180,) ANDV
(473, 1) (473,) ANSS
(473, 1) (473,) ANTM
(473, 1) (473,) AON
(473, 1) (473,) APA
(473, 1) (473,) AIV
(473, 1)

#### Trade at open 

In [1852]:
result = np.zeros(250)
k= 5

testing_dates = list(data_training_return['SPY']['Date'][-testing_length:])
for index, date in enumerate(testing_dates):
    error_map = {}
    for symbol in symbols:
        
        if symbol in reg_map and symbol !='SPY':
   

 
            try:
                mask = data_training_return['SPY']['Date'] == date
                X = data_training_return['SPY'][mask]['close_open_return'].to_numpy()
                X = np.expand_dims(X, axis=1)
                pred = reg_map[symbol].predict(X)
                mask = data_training_return[symbol]['Date'] == date
                error = data_training_return[symbol][mask]['close_open_return'].to_numpy()[0] - pred[0]
                error_map[error] = symbol
            except:
                pass
    error_list = list(error_map.items())
    error_list.sort()
 
 
    for j in range(k):
        if j< len(error_list):   
            symbol = error_list[j][1]
            mask = data_training_return[symbol]['Date'] == date
            result[index] += (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 
            
            symbol = error_list[-j-1][1]
            mask = data_training_return[symbol]['Date'] == date
            result[index] -= (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 

Performance(result, c)
            

0.003576484798460666
3.046144029128202


#### Trade at close 

In [1853]:
result = np.zeros(250)
k=5
testing_dates = list(data_training_return['SPY']['Date'][-250:])
for index, date in enumerate(testing_dates):
    error_map = {}
    for symbol in symbols:
        
        if symbol in reg_map and symbol !='SPY':
   

 
            try:
                mask = data_training_return['SPY']['Date'] == date
                X = data_training_return['SPY'][mask]['vwap_close_return']
                X = np.expand_dims(X, axis=1)
                pred = reg_map[symbol].predict(X)
                mask = data_training_return[symbol]['Date'] == date
                error = data_training_return[symbol][mask]['vwap_close_return'].to_numpy()[0] - pred[0]
                error_map[error] = symbol
            except:
                pass
    error_list = list(error_map.items())
    error_list.sort()
 
 
    for j in range(k):
        if j< len(error_list) and index+1<len(testing_dates): 
            try:
                symbol = error_list[j][1]
                mask = data_training_return[symbol]['Date'] == testing_dates[index+1]
                result[index+1] += (1/(2*k))*data_training_return[symbol][mask]['close_open_return'].to_numpy()[0] 

                symbol = error_list[-j-1][1]
                mask = data_training_return[symbol]['Date'] == testing_dates[index+1]
                result[index+1] -= (1/(2*k))*data_training_return[symbol][mask]['close_open_return'].to_numpy()[0] 
            except:
                pass

Performance(result,c)
            

0.0023529650988511224 2.291764697352613
